# MATH2970 Computer Assignment

**Author:** 510575452

In [3]:
# Importing Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.optimize import minimize

## Exploring Data

In [4]:
stocks = pd.read_csv('data/project_data.csv', parse_dates=['Date'])
stocks.head()

,Date,BHP,CBA,CSL,NAB,WBC,ANZ,MQG,WES,GMG,FMG
0,2022-07-01,33.62,81.91,260.65,24.64,17.21,19.16,151.74,38.63,17.75,13.25
1,2022-07-04,33.49,82.59,266.04,24.85,17.43,19.40,154.43,39.60,18.25,13.31
2,2022-07-05,33.57,82.39,270.05,24.77,17.43,19.33,155.85,39.82,18.08,13.50
3,2022-07-06,31.69,83.09,277.01,25.22,17.59,19.51,156.75,40.47,18.77,12.85
4,2022-07-07,32.68,84.06,277.85,25.22,17.72,19.86,157.67,41.04,18.69,13.41


## Question 1

### Part A

In [5]:
# Calculate the mean return of each stock
for stock in stocks.columns[1:]:
    stock_return = stocks.loc[:, stock].pct_change()
    stock_return_mean = stock_return.mean()
    print("Mean return for", stock, "=", round(stock_return_mean,6))

Mean return for BHP = 0.000529
Mean return for CBA = 0.000894
Mean return for CSL = 0.000294
Mean return for NAB = 0.000826
Mean return for WBC = 0.000974
Mean return for ANZ = 0.000828
Mean return for MQG = 0.000686
Mean return for WES = 0.001077
Mean return for GMG = 0.001472
Mean return for FMG = 0.001064


In [6]:
# Calculate the covariance matrix
covariant_matrix = stocks.iloc[:, 1:].pct_change()
covariant_matrix.cov()

,BHP,CBA,CSL,NAB,WBC,ANZ,MQG,WES,GMG,FMG
BHP,0.000224,0.000038,0.000004,0.000035,0.000036,0.000034,0.000074,0.000046,0.000052,0.000230
CBA,0.000038,0.000106,0.000041,0.000091,0.000090,0.000080,0.000077,0.000063,0.000075,0.000040
CSL,0.000004,0.000041,0.000123,0.000031,0.000029,0.000029,0.000045,0.000051,0.000064,0.000005
NAB,0.000035,0.000091,0.000031,0.000120,0.000099,0.000089,0.000081,0.000059,0.000066,0.000040
WBC,0.000036,0.000090,0.000029,0.000099,0.000126,0.000095,0.000079,0.000061,0.000066,0.000044
ANZ,0.000034,0.000080,0.000029,0.000089,0.000095,0.000115,0.000078,0.000058,0.000068,0.000043
MQG,0.000074,0.000077,0.000045,0.000081,0.000079,0.000078,0.000185,0.000086,0.000113,0.000096
WES,0.000046,0.000063,0.000051,0.000059,0.000061,0.000058,0.000086,0.000135,0.000113,0.000048
GMG,0.000052,0.000075,0.000064,0.000066,0.000066,0.000068,0.000113,0.000113,0.000279,0.000075
FMG,0.000230,0.000040,0.000005,0.000040,0.000044,0.000043,0.000096,0.000048,0.000075,0.000430


## Question 2

Define constraints:

$a = \textbf{e}^TC^-1\textbf{e}$


$b = \textbf{r}^TC^-1\textbf{e}$


$c = \textbf{r}^TC^-1\textbf{r}$


$d = ac-b^2$


If $C$ is invertible, the solution to


\begin{equation}
\begin{aligned}
    \min_{x \in \mathbb{R}^n} \quad & Z(x) = -tx^T r + \frac{1}{2} x^T C x \\
    \text{s.t.} \quad & x^T e = 1
\end{aligned}
\end{equation}


is $\textbf{x}(t) = \alpha + t\beta$


In [9]:
r = stocks.iloc[:, 1:].pct_change().mean()
C = covariant_matrix.cov()
e = np.ones(len(r))

C_inverse = np.linalg.inv(C)

a = e.T @ C_inverse @ e
b = r.T @ C_inverse @ e
c = r.T @ C_inverse @ r
d = a*c - b**2

alpha = (1/a) * (C_inverse @ e) 
beta = (C_inverse @ r) - (b/a * C_inverse @ e)

t_vals = np.linspace(0, 1, 100)

def objective_function(t):
    return alpha + t*beta

def portfolio_mean(t):   
    return (b + d * t) / a

def portfolio_variance(t):
    return (1 + d * t**2) / a

for t in t_vals:
    weights = objective_function(t)
    mean_ret = portfolio_mean(t)
    var = portfolio_variance(t)
    print(f"For t = {t:.2f}, Portfolio Weights: {weights}, Mean Return: {round(mean_ret, 6)}, Variance: {round(var, 6)}\n")

For t = 0.00, Portfolio Weights: [ 0.20833113  0.06319631  0.35878313  0.10614843  0.04936202  0.19993616
 -0.05492693  0.11631028 -0.03341882 -0.0137217 ], Mean Return: 0.000597, Variance: 5.9e-05

For t = 0.01, Portfolio Weights: [ 0.16766078  0.09045629  0.30267838  0.09053017  0.08270747  0.18183014
 -0.08344138  0.15324456  0.00102463  0.01330896], Mean Return: 0.000688, Variance: 6e-05

For t = 0.02, Portfolio Weights: [ 0.12699042  0.11771628  0.24657363  0.07491192  0.11605293  0.16372412
 -0.11195583  0.19017883  0.03546809  0.04033962], Mean Return: 0.000778, Variance: 6.3e-05

For t = 0.03, Portfolio Weights: [ 0.08632007  0.14497626  0.19046888  0.05929366  0.14939838  0.1456181
 -0.14047028  0.22711311  0.06991154  0.06737028], Mean Return: 0.000869, Variance: 6.7e-05

For t = 0.04, Portfolio Weights: [ 0.04564971  0.17223624  0.13436413  0.04367541  0.18274384  0.12751208
 -0.16898473  0.26404739  0.104355    0.09440094], Mean Return: 0.000959, Variance: 7.4e-05

For t = 